In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline

Loading Datasets


In [14]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('../input/amazonreviews/test.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('../input/amazonreviews/train.ft.txt.bz2')

In [16]:
train_labels[0]


1

In [17]:
train_texts[0]

'Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"'

In [18]:
train_labels=train_labels[0:500]
train_texts=train_texts[0:500]

Text Pre Processing


In [22]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [23]:
train_texts[0]

'great cd  my lovely pat has one of the great voices of her generation  i have listened to this cd for years and i still love it  when i m in a good mood it makes me feel better  a bad mood just evaporates like sugar in the rain  this cd just oozes life  vocals are jusat stuunning and lyrics just kill  one of life s hidden gems  this is a desert isle cd in my book  why she never made it big is just beyond me  everytime i play this  no matter black  white  young  old  male  female everybody says one thing  who was that singing   '

**COUNT VECTORIZER**

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)
X_test

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(
    X, train_labels, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))


Accuracy for C=0.01: 0.704
Accuracy for C=0.05: 0.752
Accuracy for C=0.25: 0.768
Accuracy for C=0.5: 0.76
Accuracy for C=1: 0.752


In [26]:
lr.predict(X_test[29])

array([1])

In [27]:
test_labels[29]

1

In [28]:
test_texts[29]

'based on the reviews here i bought one and i m glad i did   this vcr dvd was an early christmas present to myself after deciding to join the rest of the world in dvd land but not wanting to let go of my vhs movies quite yet  based on the reviews and price  and because i own a jvc tv  this was my choice  i agree with the easy set up  it s a bit awkward to work the whole tv vhs dvd selection options but i think i ve gotten the hang of it two comments  it s not intuitive and actually is pretty complicated  too many buttons on the remote but that might please others who are more technically minded than me  so i ve had to rely heavily on the how to manual  and setting the vcr timer is awkward    you can t enter the start and end time but have to scroll  unless i m missing something    but those are my only complaints  this is a good product for your     '